In [1]:
import pandas as pd
from sklearn.utils import shuffle
from sklearn import metrics
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import f1_score

In [2]:
df_Tt  = pd.read_excel('Taipei times.xlsx')
df_Tn  = pd.read_excel('Taiwan news.xlsx')
df_NYP = pd.read_excel('New York Post.xlsx')
df_Xin = pd.read_excel('新華網ok.xlsx')

In [3]:
df1 = pd.concat([df_Tn,df_NYP,df_Xin],axis=0)
df2=df1.astype('str')
df2.index=list(range(len(df2)))

In [4]:
data = df2
dataset = data[['標籤', '新聞標題']]
dataset['label'] = dataset['標籤']
dataset['label']=dataset['label'].replace('2',0)
dataset['label']=dataset['label'].replace('1',1)
del dataset['標籤']
#dataset= shuffle(dataset)
dataset=pd.DataFrame(dataset,columns=['label','新聞標題'])
dataset.head()

C:\Anaconda\python\envs\GPU\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Anaconda\python\envs\GPU\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
C:\Anaconda\python\envs\GPU\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFram

,label,新聞標題
0,0,Asian markets mixed ahead of Trump's inauguration
1,0,China's growth edges up but 2016 weakest year ...
2,0,"If Trump ends America's world leadership, who ..."
3,0,"If Trump ends America's world leadership, who ..."
4,0,'America First' doesn't have to mean 'American...


In [5]:
dataset['label']=dataset['label'].astype('int')

In [6]:
import multiprocessing

num_cores = multiprocessing.cpu_count()#計算可以使用多少內核來確定worker的數量
num_cores

12

In [7]:
#P.261 對每個文檔裡的每個word做分解，每個sample得到一個nx300的word2vector合集
#並合併每個word的word_vectors為sample_vecs(文檔的vector)，此時為每個文檔裡的所有word_vectors
#再把sample_vecs合併成vectorized_data，此時包含所有文檔的每個word的vector(頻率含意)
from nltk.tokenize import TreebankWordTokenizer

def tokenize_and_vectorize(dataset): 
    tokenizer = TreebankWordTokenizer() 
    vectorized_data = [] 
    expected = [] 
    for sample in dataset:#.iloc[:,1]: 
        tokens = tokenizer.tokenize(sample[1])#抓出sample的文字部分做分解
            
        vectorized_data.append(tokens) 
        
    return vectorized_data
token_list = tokenize_and_vectorize(dataset.values)
token_list

[['Asian', 'markets', 'mixed', 'ahead', 'of', 'Trump', "'s", 'inauguration'],
 ['China',
  "'s",
  'growth',
  'edges',
  'up',
  'but',
  '2016',
  'weakest',
  'year',
  'since',
  '1990'],
 ['If',
  'Trump',
  'ends',
  'America',
  "'s",
  'world',
  'leadership',
  ',',
  'who',
  'will',
  'step',
  'up',
  '?'],
 ['If',
  'Trump',
  'ends',
  'America',
  "'s",
  'world',
  'leadership',
  ',',
  'who',
  'will',
  'step',
  'up',
  '?'],
 ["'America",
  'First',
  "'",
  'does',
  "n't",
  'have',
  'to',
  'mean',
  "'American",
  'Only',
  "'",
  '401k'],
 ['US-Mexico',
  'trade',
  'war',
  'could',
  'hit',
  'Mexico',
  'economy',
  ',',
  'spur',
  'migration'],
 ['Business', 'Highlights'],
 ['Which',
  'US',
  'metro',
  'areas',
  'depend',
  'most',
  'on',
  'exports',
  '-',
  'at',
  'a',
  'glance'],
 ['Why',
  'the',
  'US',
  'trade',
  'gap',
  'could',
  'obstruct',
  'Trump',
  "'s",
  'economic',
  'vision'],
 ['Trump',
  "'s",
  'contempt',
  'for',
  'trade

In [8]:
#產生一個token_list的Word2Vec
from gensim.models import Word2Vec

num_features = 300#word vector的向量元素的數量
min_word_count = 2#模型中要考慮的最小字數。語料庫很小，減少；語料庫很大，增加
num_workers = num_cores#train CPU的內核數
window_size = 6#上下文窗口大小
subsampling =1e-3#2-3#常用詞的二次採樣率

model = Word2Vec(token_list,
                 workers=num_workers, 
                 size=num_features, 
                 min_count=min_word_count, 
                 window=window_size, 
                 sample=subsampling)

In [9]:
from gensim.models import KeyedVectors

#word_vectors = KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin', binary=True)
word_vectors =model

In [10]:
#P.261 對每個文檔裡的每個word做分解，每個sample得到一個nx300的word2vector合集
#並合併每個word的word_vectors為sample_vecs(文檔的vector)，此時為每個文檔裡的所有word_vectors
#再把sample_vecs合併成vectorized_data，此時包含所有文檔的每個word的vector(頻率含意)
from nltk.tokenize import TreebankWordTokenizer

def tokenize_and_vectorize(dataset): 
    tokenizer = TreebankWordTokenizer() 
    vectorized_data = [] 
    expected = [] 
    for sample in dataset:#.iloc[:,1]: 
        tokens = tokenizer.tokenize(sample[1])#抓出sample的文字部分做分解
        sample_vecs = [] 
        for token in tokens: 
            try: 
                sample_vecs.append(word_vectors[token])
                #word_vectors[token]每個字在word_vectors的vector(頻率含意) 
                
            except KeyError: 
                pass # No matching token in the Google w2v vocab 
            
        vectorized_data.append(sample_vecs) 
        
    return vectorized_data

In [11]:
vectorized_data = tokenize_and_vectorize(dataset.values)
expected = dataset.label

C:\Anaconda\python\envs\GPU\lib\site-packages\ipykernel_launcher.py:15: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  from ipykernel import kernelapp as app


In [12]:
pd.DataFrame(vectorized_data[0])

,0,1,2,3,4,5,6,7,8,9,...,290,291,292,293,294,295,296,297,298,299
0,0.038391,-0.073282,0.104131,-0.069792,0.064530,-0.088715,-0.026647,0.026561,0.021252,-0.042576,...,0.028419,-0.044298,-0.118794,0.068466,0.105299,0.011387,0.043955,0.132468,0.023011,-0.088251
1,0.029671,-0.052580,0.076311,-0.050398,0.048552,-0.061872,-0.019550,0.019074,0.014759,-0.031350,...,0.021471,-0.031893,-0.087524,0.049458,0.076240,0.007359,0.031731,0.093562,0.016195,-0.064943
2,0.019651,-0.036261,0.053769,-0.034577,0.034820,-0.043328,-0.015641,0.011899,0.010705,-0.022215,...,0.015911,-0.022493,-0.062350,0.034336,0.053313,0.004076,0.020864,0.068742,0.009400,-0.047040
3,0.011890,-0.020696,0.033472,-0.022029,0.018742,-0.027840,-0.007086,0.009152,0.008306,-0.012108,...,0.010575,-0.012809,-0.036758,0.020059,0.032077,0.002506,0.012705,0.039732,0.007033,-0.027854
4,0.081190,-0.145261,0.217532,-0.139863,0.135969,-0.181837,-0.058270,0.052109,0.046140,-0.091020,...,0.062531,-0.092975,-0.250192,0.137822,0.214389,0.023673,0.086439,0.271848,0.045402,-0.182162
5,0.066582,-0.118755,0.176103,-0.114203,0.110348,-0.148343,-0.045910,0.042864,0.038904,-0.073919,...,0.049851,-0.077394,-0.201626,0.113177,0.176681,0.019023,0.071497,0.220097,0.033891,-0.148885
6,0.071191,-0.130922,0.192997,-0.123522,0.118560,-0.161903,-0.052033,0.047802,0.040048,-0.081285,...,0.055517,-0.082298,-0.220930,0.122635,0.190760,0.018782,0.075481,0.238638,0.037532,-0.160088


In [13]:
#切割資料
#split_point = int(len(vectorized_data)*.8)
#split_point_end = int(len(vectorized_data)*.5)

#x_train = vectorized_data[:split_point] 
x_train=vectorized_data[941:1411]+vectorized_data[1647:2117]
y_train =pd.concat([expected[941:1411],expected[1647:2117]])


#x_test = vectorized_data[split_point:-(int((len(vectorized_data)-split_point)*0.5))]#split_point_end]
#y_test = expected[split_point:-(int((len(vectorized_data)-split_point)*0.5))]#split_point_end]


In [14]:
test_T=vectorized_data[:705]
#test_T.index=range(len(test_T))
test_U=vectorized_data[1412:1647]
#test_U.index=range(len(test_U))
test_C=vectorized_data[1412:1647]+vectorized_data[1882:2117]
#test_C.index=range(len(test_C))

In [15]:
y_test_T=expected[:705]
#y_test_T.index=range(len(test_T))
y_test_U=pd.concat([expected[1412:1647]])

#y_test_U.index=range(len(test_U))
y_test_C=pd.concat([expected[1412:1647],expected[1882:2117]])
#y_test_C.index=range(len(test_C))

In [16]:
#P.289 發現token長度設得太高，可調降長度至平均文檔token長度(Avg length)
def test_len(data, maxlen): 
    total_len = truncated = exact = padded = 0 
    for sample in data: 
        total_len += len(sample) 
        if len(sample) > maxlen: 
            truncated += 1 
        elif len(sample) < maxlen: 
            padded += 1 
        else: 
            exact +=1 
    print('Padded: {}'.format(padded)) 
    print('Equal: {}'.format(exact)) 
    print('Truncated: {}'.format(truncated)) 
    print('Avg length: {}'.format(total_len/len(data)))
    Avg_length = total_len/len(data)

In [17]:
test_len(vectorized_data, 600)

Padded: 2118
Equal: 0
Truncated: 0
Avg length: 9.205854579792257


In [18]:
#network的參數
maxlen = 400 #token的長度
batch_size = 32 #在反向傳播錯誤和更新權重之前要顯示多少個樣本
embedding_dims = 300 #token的word_vectors長度
epochs = 2 #整個訓練數據集通過網絡的次數

In [19]:
#P.262 padding(填充)不夠的data或truncate(截斷)多出的data
def pad_trunc(data, maxlen): 
    """ For a given dataset pad with zero vectors or truncate to maxlen """ 
    new_data = []
    # Create a vector of 0s the length of our word vectors 
    zero_vector = [] 
    for _ in range(len(data[0][0])):#每個文檔裡每個word的word_vectors(分數含意)長度都是300  
        zero_vector.append(0.0)#生成一個長度為300的zero vectors
        
    for sample in data:#每個文檔裡的字 
        if len(sample) > maxlen:#字的長度大於maxlen=500
            temp = sample[:maxlen] #只擷取到maxlen
        elif len(sample) < maxlen:
            temp = sample 
            # Append the appropriate number 0 vectors to the list 
            additional_elems = maxlen - len(sample) #不足500的長度
            for _ in range(additional_elems): 
                temp.append(zero_vector) #把不足的字的長度以zero vectors加上
        else: 
            temp = sample 
        new_data.append(temp) 
    return new_data

In [20]:
import numpy as np

x_train = pad_trunc(x_train, maxlen)
test_U = pad_trunc(test_U, maxlen)
test_C = pad_trunc(test_C, maxlen)
test_T = pad_trunc(test_T, maxlen)
#轉為numpy array供Keras使用
#np.reshape(資料，(幾個文檔，token的長度，word_vector的長度))
x_train = np.reshape(x_train,(len(x_train), maxlen, embedding_dims))
y_train = np.array(y_train) 
test_U = np.reshape(test_U,(len(test_U), maxlen, embedding_dims))
y_test_U = np.array(y_test_U)
test_T = np.reshape(test_T,(len(test_T), maxlen, embedding_dims))
y_test_T= np.array(y_test_T)
test_C = np.reshape(test_C,(len(test_C), maxlen, embedding_dims))
y_test_C = np.array(y_test_C)



# RNN

In [21]:
# 建模-使用遞歸神經網路(RNN)
#考量token的順序，一次放入一個token，試圖記住過去的output，看step t對step t+1的影響
#以此類推到文檔的最後一個token
from keras.models import Sequential 
from keras.layers import Dense, Dropout, Flatten, SimpleRNN

num_neurons = 50 #hidden neuron
model = Sequential()
model.add(SimpleRNN( 
    num_neurons, return_sequences=True,#return the network value at each time step
    input_shape=(maxlen, embedding_dims)))
model.add(Dropout(.2))#隨機關閉將傳到Dropout layer的輸入的百分比，減少學習特定的訓練集，免overfitting
model.add(Flatten()) #Dense layer需要一個由n個元素組成的“flat” vector作為input，因此使用Flatten()
model.add(Dense(1, activation='sigmoid'))#Dense layer，值在0(低於0.5)到1(高於0.5)之間#sigmoid
#model.add(Dense(1, activation='softmax'))
model.compile('rmsprop', 'binary_crossentropy', metrics=['accuracy'])
#因有打開或關閉的輸出神經元，所以使用binary_crossentropy
model.summary()#檢查模型的內部

Using TensorFlow backend.


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
simple_rnn_1 (SimpleRNN)     (None, 400, 50)           17550     
_________________________________________________________________
dropout_1 (Dropout)          (None, 400, 50)           0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 20000)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 20001     
Total params: 37,551
Trainable params: 37,551
Non-trainable params: 0
_________________________________________________________________


In [22]:
model.fit(x_train, y_train, 
           batch_size=batch_size, 
           epochs=epochs, 
           validation_data=(test_U, y_test_U)) 

Train on 940 samples, validate on 235 samples
Epoch 1/2
940/940 [==============================] - 4s 4ms/step - loss: 0.9910 - accuracy: 0.5074 - val_loss: 0.9626 - val_accuracy: 0.2128
Epoch 2/2
940/940 [==============================] - 4s 4ms/step - loss: 0.8568 - accuracy: 0.4968 - val_loss: 0.7921 - val_accuracy: 0.4936


In [23]:
 from sklearn.metrics import classification_report

In [24]:
y_pred=model.predict_classes(test_U)
y_true=y_test_U
print(accuracy_score(y_true, y_pred))
print(metrics.precision_score(y_true, y_pred, average='binary'))
print(metrics.recall_score(y_true, y_pred, average='binary'))
print(metrics.f1_score(y_true, y_pred, average='binary') )
print(y_pred.sum())
print(y_true.sum())

0.49361702127659574
0.0
0.0
0.0
119
0


C:\Anaconda\python\envs\GPU\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# LSTM

In [ ]:
from keras.models import Sequential 
from keras.layers import Dense, Dropout, Flatten, LSTM 

num_neurons = 50 
model = Sequential() 
model.add(LSTM(num_neurons, return_sequences=True, input_shape=(maxlen, embedding_dims))) 
model.add(Dropout(.2)) 
model.add(Flatten()) 
model.add(Dense(1, activation='sigmoid')) 
#model.add(Dense(1, activation='softmax')) 
model.compile('rmsprop', 'binary_crossentropy', metrics=['accuracy']) 
model.summary() 

In [ ]:
model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_data=(test_C, y_test_C)) 

In [ ]:
y_pred=model.predict_classes(test_U)
y_true=y_test_U
print(accuracy_score(y_true, y_pred))
print(metrics.precision_score(y_true, y_pred, average='binary'))
print(metrics.recall_score(y_true, y_pred, average='binary'))
print(metrics.f1_score(y_true, y_pred, average='binary') )